In [ ]:
import json
import random
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))
collaborator_news = random.randint(20, 90)
manager_news = random.randint(20, 90)


def add_entire_structure_with_manager(tx, entreprise_name, direction_manager, service_manager, equipe_manager, direction_name, service_name, equipe_name, first_name, last_name, age, nb_news, email):
    # This Cypher query creates nodes for all entities and relationships between them
    query = """
    MERGE (entreprise:Entreprise {nom: $entreprise_name})
    MERGE (entreprise)-[:HAS_DIRECTION]->(direction:Direction {nom: $direction_name})
    MERGE (direction)-[:HAS_SERVICE]->(service:Service {nom: $service_name})
    MERGE (service)-[:HAS_EQUIPE]->(equipe:Equipe {nom: $equipe_name})
    MERGE (equipe)-[:HAS_MANAGER]->(equipe_manager:Manager {prenom: $equipe_manager_prenom, nom: $equipe_manager_nom, email: $equipe_manager_email, nb_news: $equipe_manager_nb_news})
    MERGE (direction)-[:HAS_MANAGER]->(direction_manager:Manager {prenom: $direction_manager_prenom, nom: $direction_manager_nom, email: $direction_manager_email, nb_news: $direction_manager_nb_news})
    MERGE (service)-[:HAS_MANAGER]->(service_manager:Manager {prenom: $service_manager_prenom, nom: $service_manager_nom, email: $service_manager_email, nb_news: $service_manager_nb_news})
    MERGE (employee:Collaborateur {prenom: $first_name, nom: $last_name, age: $age, email: $email, nb_news: $employee_nb_news})
    MERGE (equipe_manager)-[:MANAGES_EQUIPE]->(equipe)
    MERGE (direction_manager)-[:MANAGES_DIRECTION]->(direction)
    MERGE (service_manager)-[:MANAGES_SERVICE]->(service)
    MERGE (equipe)-[:HAS_EMPLOYEE]->(employee)
    """
    tx.run(query, entreprise_name=entreprise_name, direction_manager_prenom=direction_manager['prenom'], direction_manager_nom=direction_manager['nom'], direction_manager_email=direction_manager['email'], direction_manager_nb_news=direction_manager['nb_news'], service_manager_prenom=service_manager['prenom'], service_manager_nom=service_manager['nom'], service_manager_email=service_manager['email'], service_manager_nb_news=service_manager['nb_news'], equipe_manager_prenom=equipe_manager['prenom'], equipe_manager_nom=equipe_manager['nom'], equipe_manager_email=equipe_manager['email'], equipe_manager_nb_news=equipe_manager['nb_news'], direction_name=direction_name, service_name=service_name, equipe_name=equipe_name, first_name=first_name, last_name=last_name, age=age, email=email, employee_nb_news=nb_news)

def add_manager_par_relation(tx, direction_manager_nom, direction_manager_prenom, manager_par_nom, manager_par_prenom):
# This Cypher query creates a "MANAGER_PAR" relationship between two managers
    query = """
    MATCH (m1:Manager {nom: $direction_manager_nom, prenom: $direction_manager_prenom})
    MATCH (m2:Manager {nom: $manager_par_nom, prenom: $manager_par_prenom})
    MERGE (m1)-[:MANAGER_PAR]->(m2)
"""
    tx.run(query, direction_manager_nom=direction_manager_nom, direction_manager_prenom=direction_manager_prenom, manager_par_nom=manager_par_nom, manager_par_prenom=manager_par_prenom)

with open("C:/Users/ESPRI/Desktop/PFE/Neo4j/nom_prenom.json") as fichier_test1:
    data1 = json.load(fichier_test1)

with open('C:/Users/ESPRI/Desktop/PFE/Neo4j/Entreprise.json') as f:
    data = json.load(f)
for entreprise in data:
    entreprise_name = entreprise['nom']
    used_names = set()  # Set to keep track of used names
    used_emails = set()  # Set to keep track of used emails

    for direction in entreprise['children']:
        direction_name = direction['nom']
        
        # Randomly choose a manager for the direction
        aleatoire_direction_manager_prenom = random.choice(data1['prenom'])
        aleatoire_direction_manager_nom = random.choice(data1['nom'])
        aleatoire_direction_manager_email = aleatoire_direction_manager_prenom.lower() + '.' + aleatoire_direction_manager_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

        direction_manager = {'prenom': aleatoire_direction_manager_prenom, 'nom': aleatoire_direction_manager_nom, 'email': aleatoire_direction_manager_email, 'nb_news': manager_news}

        # Add the direction manager to the used names and emails sets
        used_names.add((direction_manager['prenom'], direction_manager['nom']))
        used_emails.add(direction_manager['email'])

        for service in direction['children']:
            service_name = service['nom']

            # Randomly choose a manager for the service
            aleatoire_service_manager_prenom = random.choice(data1['prenom'])
            aleatoire_service_manager_nom = random.choice(data1['nom'])
            aleatoire_service_manager_email = aleatoire_service_manager_prenom.lower() + '.' + aleatoire_service_manager_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

            service_manager = {'prenom': aleatoire_service_manager_prenom, 'nom': aleatoire_service_manager_nom, 'email': aleatoire_service_manager_email, 'nb_news': manager_news}

            # Add the service manager to the used names and emails sets
            used_names.add((service_manager['prenom'], service_manager['nom']))
            used_emails.add(service_manager['email'])

            for equipe in service['children']:
                equipe_name = equipe['nom']

                # Randomly choose a manager for the team
                aleatoire_equipe_manager_prenom = random.choice(data1['prenom'])
                aleatoire_equipe_manager_nom = random.choice(data1['nom'])
                aleatoire_equipe_manager_email = aleatoire_equipe_manager_prenom.lower() + '.' + aleatoire_equipe_manager_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

                equipe_manager = {'prenom': aleatoire_equipe_manager_prenom, 'nom': aleatoire_equipe_manager_nom, 'email': aleatoire_equipe_manager_email, 'nb_news': manager_news}

                # Add the team manager to the used names and emails sets
                used_names.add((equipe_manager['prenom'], equipe_manager['nom']))
                used_emails.add(equipe_manager['email'])

                for i in range(5):
                    # Randomly choose a collaborator and generate a unique email
                    aleatoire_collaborateur_prenom = random.choice(data1['prenom'])
                    aleatoire_collaborateur_nom = random.choice(data1['nom'])
                    aleatoire_collaborateur_email = aleatoire_collaborateur_prenom.lower() + '.' + aleatoire_collaborateur_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

                    # Check if the generated email is not already used
                    while aleatoire_collaborateur_email in used_emails:
                        aleatoire_collaborateur_prenom = random.choice(data1['prenom'])
                        aleatoire_collaborateur_nom = random.choice(data1['nom'])
                        aleatoire_collaborateur_email = aleatoire_collaborateur_prenom.lower() + '.' + aleatoire_collaborateur_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

                    # Add the collaborator email to the used emails set
                    used_emails.add(aleatoire_collaborateur_email)

                    # Randomly generate the number of news for the collaborator
                    aleatoire_collaborateur_nb_news = random.randint(0, collaborator_news)

                    # Add the collaborator to the graph
                    with driver.session() as session:
                        session.execute_write(add_entire_structure_with_manager, entreprise_name, direction_manager, service_manager, equipe_manager, direction_name, service_name, equipe_name, aleatoire_collaborateur_prenom, aleatoire_collaborateur_nom, 30, aleatoire_collaborateur_nb_news, aleatoire_collaborateur_email)

                # Create "MANAGER_PAR" relationships between managers
                with driver.session() as session:
                    session.execute_write(add_manager_par_relation, direction_manager['nom'], direction_manager['prenom'], service_manager['nom'], service_manager['prenom'])
                    session.execute_write(add_manager_par_relation, service_manager['nom'], service_manager['prenom'], equipe_manager['nom'], equipe_manager['prenom'])

In [ ]:
import json
from tabulate import tabulate
import random
import math
from datetime import datetime

# Ouvrir le fichier JSON des mois
with open("C:/Users/ESPRI/Desktop/PFE/Neo4j/Moyen_mois.json") as f:
    data = json.load(f)

# Ouvrir le fichier JSON des heures
with open("C:/Users/ESPRI/Desktop/PFE/Neo4j/heures.json") as f:
    data1 = json.load(f)
    
data1["Heures"]=list(range(24))
    
# Convertir les données en liste de listes pour le tabulateur
print("Tableau initial des mois avec leurs moyennes")
table_mois = []
for i in range(len(data["Mois"])):
    table_mois.append([data["Mois"][i], data["Moyen"][i]])

    
# Afficher le tableau avec tabulate
print(tabulate(table_mois, headers=["Mois", "Moyenne"]))
    
# print("Le tableau  ")
# T=[]
# moyenne_du_jour = []
# for i in range(len(data["Mois"])):
#    moyenne = data["Moyen"][i]
#    moyenne_du_jour = moyenne + random.randint(-2, 2)
#    mois_new=data["Mois"][i]
#    T.append([mois_new,moyenne_du_jour])
# Afficher le résultat
# print(tabulate(T, headers=["un jour du mois", "Moyenne"]))



table_initiale_heures = []

for i in range(len(data1["Heures"])):
    table_initiale_heures.append([data1["Heures"][i], data1["Pourcentage"][i]])
print(tabulate(table_initiale_heures, headers=["Heures", "Pourcentage"]))

# Créer une copie du tableau initial
table_copy_heures = table_initiale_heures.copy()

# Trier aléatoirement la copie du tableau
random.shuffle(table_copy_heures)

# Afficher la copie triée aléatoirement avec tabulate
print(tabulate(table_copy_heures, headers=["Heures", "Pourcentage"]))



def rand_round (num):
    a = random.randint(0, 1)
    if(a==1):
        return math.ceil(num)
    else :
        return math.floor(num)
    
def get_month_from_date(date):
    month = date.month
    return month

# cette fonction, donne pour une date donnée, le nb de publications chaque heure dans un tableau
def plublications_par_heure(date):
    
    #nb de publications par heure
    resultats = []
    
    i = get_month_from_date(date) - 1
    mois = table_mois[i][0]
    moyenne_mois = table_mois[i][1]
    #print("moyenne du mois", moyenne_mois)
    rand_variation = random.randint(-2, 2)
    #print("variation aléatoire", rand_variation)
    nb_pub = rand_round( moyenne_mois + rand_variation)
    print("nombre de publications pour cette journée: ",nb_pub)
    reste = nb_pub
    for j in range(len(table_copy_heures)):
        heure = table_copy_heures[j][0]
        pourcentage = table_copy_heures[j][1]
        resultat = rand_round((nb_pub * pourcentage / 100))  # Calculer le résultat arrondi à l'entier le plus proche
        if resultat <= reste :
            resultats.append([heure, resultat])
            reste = reste - resultat
        else :
            resultats.append([heure, reste])
            reste = 0
    if reste != 0:
        resultats[len(resultats)-1][1] += reste

    sortedresults = sorted(resultats, key=lambda x: x[0])
    
    finalresults = [x[1] for x in sortedresults]
    
    # Afficher le tableau final results
    #print(finalresults)
    
    return finalresults


plublications_par_heure(datetime(2023, 6, 18))


In [ ]:
import json
import datetime
import random
from neo4j import GraphDatabase
from datetime import datetime
from datetime import timedelta

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))

collaborateurs = []
query = """MATCH (c:Collaborateur)
WITH c, RAND() AS r
ORDER BY r
LIMIT 1
RETURN c"""

#def random_datetime(start_date, end_date):
#    start_timestamp = start_date.timestamp()
#   end_timestamp = end_date.timestamp()
#   random_timestamp = random.uniform(start_timestamp, end_timestamp)
#    random_date = datetime.datetime.fromtimestamp(random_timestamp)
#    return random_date

def get_collab():
     with driver.session() as session:
        result = session.run(query)
        collab = result.single()["c"]  # Utilisation de la méthode single() pour obtenir le collaborateur unique
        print(collab["prenom"],collab["nom"])
        return collab
#    for record in result:
#        collaborateur = record["c"]
#        collaborateurs.append(collaborateur)

# Charger les dates de début et de fin à partir du fichier JSON
with open("/Users/ESPRI/Desktop/PFE/Neo4j/date.json", "r") as file:
    dates_data = json.load(file)
date_debut_str = dates_data["date_debut"]
date_fin_str = dates_data["date_fin"]

# Convertir les dates de chaînes de caractères en objets datetime
date_debut = datetime.strptime(date_debut_str, "%Y-%m-%d")
date_fin = datetime.strptime(date_fin_str, "%Y-%m-%d")

def iterate_datetime(start_datetime, end_datetime):
    current_datetime = start_datetime
    onehour = timedelta(hours=1)

    while current_datetime <= end_datetime:
        pub_per_h = plublications_par_heure(current_datetime)
        for h in range(24): 
            formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M")
            current_hour = current_datetime.hour
            print(formatted_datetime,  pub_per_h[current_hour])
            current_datetime += onehour
        
iterate_datetime(date_debut,date_fin)
        
# Fermer la connexion à la base de données Neo4j
driver.close()

""""

# Convertir les dates de chaînes de caractères en objets datetime
date_debut = datetime.datetime.strptime(date_debut_str, "%Y-%m-%d")
date_fin = datetime.datetime.strptime(date_fin_str, "%Y-%m-%d")

# ...
# Créer une copie du tableau résultat
tableau_resultat_copy = resultats.copy()
print(tableau_resultat_copy)
# Trier aléatoirement la copie du tableau résultat
random.shuffle(tableau_resultat_copy)

# Liste pour stocker les publications
publications = []

# Variable pour l'incrémentation de l'identifiant de publication
publication_id = 1

# Parcourir chaque élément du tableau résultat trié aléatoirement
for i in range(len(tableau_resultat_copy)):

    mois = tableau_resultat_copy[i][0]
    heures = tableau_resultat_copy[i][1]
    resultat = tableau_resultat_copy[i][2]

    objectif_atteint = True  # Indicateur pour vérifier si l'objectif est atteint pour tous les intervalles d'heures

    # Parcourir chaque heure dans l'intervalle d'heures
    for heure in heures:
        nb_publications = 0  # Compteur du nombre de publications créées pour cette heure

        while nb_publications < resultat:
            # Sélectionner un collaborateur au hasard
            collaborateur = random.choice(collaborateurs)

            # Générer une valeur aléatoire entre 1 et 100
            valeur_aleatoire = random.randint(1, 100)

            # Comparer la valeur aléatoire avec le nb_news du collaborateur
            if valeur_aleatoire <= collaborateur["nb_news"]:
                # Le collaborateur a créé une publication

                # Générer une date de publication aléatoire entre la date de début et la date de fin
                random_date = random_datetime(date_debut, date_fin)

                # Générer le nom propre de la publication en utilisant l'identifiant de publication
                nom_propre = f"Publication {publication_id}"

                # Ajouter la publication à la liste des publications avec l'identifiant et le nom propre
                publication = {
                    "id": publication_id,
                    "nom_propre": nom_propre,
                    "date": random_date.strftime("%Y-%m-%dT%H:%M:%S"),
                    "auteur": f"{collaborateur['prenom']} {collaborateur['nom']}"
                }
                publications.append(publication)

                # Créer la publication dans la base de données Neo4j
                with driver.session() as session:
                    session.run(
                        "CREATE (p:Publication {id: $publication_id, nom_propre: $nom_propre, date: datetime($date), auteur: $auteur})",
                        publication_id=publication_id,
                        nom_propre=nom_propre,
                        date=random_date.strftime("%Y-%m-%dT%H:%M:%S"),
                        auteur=f"{collaborateur['prenom']} {collaborateur['nom']}"
                    )

                    # Créer la relation entre la publication et son collaborateur créateur
                    session.run(
                        "MATCH (p:Publication {id: $publication_id}), "
                        "(c:Collaborateur {prenom: $prenom, nom: $nom}) "
                        "CREATE (p)-[:CREE_PAR]->(c)",
                        publication_id=publication_id,
                        prenom=collaborateur['prenom'],
                        nom=collaborateur['nom']
                    )

                # Incrémenter le nombre de publications et l'identifiant de publication
                nb_publications += 1
                publication_id += 1

            # Vérifier si l'objectif est atteint pour cette heure
            if nb_publications == resultat:
                break

        # Vérifier si l'objectif est atteint pour cet intervalle d'heures
        if nb_publications != resultat:
            objectif_atteint = False
            break  # Sortir de la boucle si l'objectif n'est pas atteint pour cet intervalle d'heures

    # Vérifier si l'objectif est atteint pour tous les intervalles d'heures
    if objectif_atteint:
        break  # Sortir de la boucle si l'objectif est atteint pour tous les intervalles d'heures

# ...

publications_triees = sorted(publications, key=lambda x: datetime.datetime.strptime(x['date'], "%Y-%m-%dT%H:%M:%S"))

# Enregistrer les publications triées dans un fichier texte
with open("C:/Users/ESPRI/Desktop/PFE/Neo4j/publication.txt", "w") as file:
    for publication in publications_triees:
        file.write(f"{publication['nom_propre']}:")
        file.write(f" Date: {publication['date']}")
        file.write(f" Auteur: {publication['auteur']}\n")
        file.write("\n")

print("Les publications triées ont été enregistrées dans le fichier 'publications.txt'.")
"""